In [18]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

openai_client = AzureOpenAI(
  api_key = os.environ["AOAI_API_KEY"],  
  api_version = os.environ["AOAI_API_VERSION"],
  azure_endpoint = os.environ["AOAI_ENDPOINT"]
)

search_client = SearchClient(
    endpoint= os.environ["SEARCH_SERVICE_ENDPOINT"],
    index_name="docs3",
    credential=AzureKeyCredential(os.environ["SEARCH_QUERY_KEY"])
)

prompt= "就業時間に関してどのような規定があるのか重要度順に3つ教えてください"

def generate_embeddings(prompt, model="text-embedding-3-small-deploy"): # model = "deployment_name"
    response = openai_client.embeddings.create(input = prompt, model=model).data[0].embedding
    return response
    # print(response[:500])

# response = generate_embeddings(prompt)

vector_query = VectorizedQuery(
        vector=generate_embeddings(prompt),
        k_nearest_neighbors=3,  # 上位3件の結果を取得します
        fields="contentVector" # ベクトル検索を行うフィールドを指定します
    )

results = search_client.search(
    search_text='', # ベクトル検索のみ行うためテキストクエリは空
    vector_queries = [vector_query],
    select=['id', 'content'],
)

# print(results)
# semantic_answers = results.get_answers()
# print(semantic_answers)

# for result in results:
#     print(result["content"])

# print(result)
# 最初の検索結果を取得
first_result = next(results, None)
print(first_result["content"])

す。）は、１週４４時間まで働かせることが認められています（労基法第４０条、労基則
第２５条の２）。 
また、労基法第３２条第２項において、１日の労働時間の上限は８時間と定められて
います。 
３ 休憩時間については、１日の労働時間が６時間を超える場合には少なくとも４５分、
８時間を超える場合には少なくとも１時間の休憩時間を与えなければなりません（労基
法第３４条）。 
４ 休日については、毎週少なくとも１回又は４週間を通じ４日以上与えなければなりま
せん（労基法第３５条）。 
５ 上記２から４までの労基法の規定に適合する労働条件とするためには、①週休２日制
とする、②週休１日制で１日の所定労働時間を短く設定する、③変形労働時間制（１か
月単位、１年単位等）を導入する等の方法がありますので、それぞれの事業場の実情に
応じて、下記の規程例を参考に就業規則を作成してください。 
 
 
 
［例１］ 完全週休２日制を採用する場合の規程例 
 
１日の労働時間を８時間とし、完全週休２日制を採用する場合の規程例です。 
 
(労働時間及び休憩時間)  
第１９条  労働時間は、１週間については４０時間、１日については８時間とする。 
２ 始業・終業の時刻及び休憩時間は、次のとおりとする。ただし、業務の都合その他
やむを得ない事情により、これらを繰り上げ、又は繰り下げることがある。この場合、   
前日までに労働者に通知する。 
 
 - 23 - 
 ① 一般勤務 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
② 交替勤務 
 （イ）１番（日勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
（ロ）２番（準夜勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
（ハ）３番（夜勤） 
始業・終業時刻 休憩時間 
始業  午前  時  分 
  時  分から  時  分まで 
終業  午後  時  分 
 
３ 交替勤務における各労働者の勤務は、別に定めるシフト表により、前月の   日
までに各労働者に通知する。


In [19]:
openai_client = AzureOpenAI(
  api_key = os.environ["AOAI_API_KEY"],  
  api_version = os.environ["AOAI_API_VERSION"],
  azure_endpoint = os.environ["AOAI_ENDPOINT"]
)

# AIのキャラクターを決めるためのシステムメッセージを定義する。
system_message_chat_conversation = """
あなたはユーザーの質問に回答するチャットボットです。
回答については、「Sources:」以下に記載されている内容に基づいて回答してください。
回答は簡潔にしてください。
「Sources:」に記載されている情報以外の回答はしないでください。
また、ユーザーの質問に対して、Sources:以下に記載されている内容に基づいて適切な回答ができない場合は、「すみません。わかりません。」と回答してください。
回答の中に情報源の提示は含めないでください。例えば、回答の中に「Sources:」という形で情報源を示すことはしないでください。
"""

messages_for_vector_answer = []

messages_for_vector_answer.append({"role": "system", "content": system_message_chat_conversation})
user_message = """
{query}

Sources:
{source}
""".format(query=prompt, source=first_result["content"])

messages_for_vector_answer.append({"role": "user", "content": user_message})

# Azure OpenAI Serviceに回答生成を依頼する。
response = openai_client.chat.completions.create(
    model="gpt-35-turbo-deploy",
    messages=messages_for_vector_answer
)
response_text = response.choices[0].message.content
print(response_text)

1. 1週間の労働時間は労働基準法で44時間までと定められています。
2. 1日の労働時間の上限は、労働基準法で8時間までと定められています。
3. 労働時間が6時間を超える場合、休憩時間を少なくとも45分、8時間を超える場合は少なくとも1時間の休憩時間を与えなければなりません。
